## SCARR for Tiled Data (EM Measurements)

In [ ]:
import zarr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset_name = "15x8x20000_r1_singlerail5_sr_ise_compressed.zarr"

dataset = zarr.open(dataset_name, "r")

In [ ]:
dataset.tree()

In [ ]:
import matplotlib.pyplot as plt
print('Raw Plot')
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(dataset["0/0/traces"][0,:], color='red')

ax.set_xlabel('Samples')
ax.set_ylabel('ADC Value')
plt.show()

Tile tuple generator

In [ ]:
def create_list_of_tuples(list_x, list_y):
    result = []  # Empty list to store the tuples
    for i in range(len(list_x)):
        # Create a tuple from corresponding elements
        for j in range(len(list_y)):
            tuple_element = (list_x[i], list_y[j])
            result.append(tuple_element)  # Append the tuple to the list
    return result

In [ ]:
all_tiles = create_list_of_tuples(np.arange(0,15),np.arange(0,8))

SNR Calculation

In [ ]:
from scarr.engines.snr import SNR as snr
from scarr.file_handling.trace_handler import TraceHandler as th
from scarr.container.container import Container, ContainerOptions

#handler = th(fileName='15x8x20000_r1_singlerail5_sr_ise.zarr', batchStart=10000)
handler = th(fileName='15x8x20000_r1_singlerail5_sr_ise_compressed.zarr')
engine = snr()
container = Container(options=ContainerOptions(engine=engine, handler=handler), tile_positions=all_tiles)

In [ ]:
# this step computes the SNR for 120 EM tiles (15x8) in approx. 45s on the i7-11800H
engine.run(container)

In [ ]:
results = engine.get_result()

In [ ]:
results.shape

Let's do a simple 2D plot just for the purpose of demonstration (not much will be visible).

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 4))
#plt.yscale('log')
#ax.plot(results[0,0,:2000], color='red')
ax.plot(results[0,0,:], color='red')
ax.plot(results[1,0,:], color='green')

ax.set_xlabel('Samples')
ax.set_ylabel('SNR')
plt.show()

Much more useful will be the heatmap plot of the SNR to identify the best tiles. The left half of the EM map will be mostly dark, whereas the right half will show much brighter colors, representative for a much higher SNR.

In [ ]:
#def flat_to_shaped_tile_result(tile_results, dim_x, dim_y):
dim_x = 15
dim_y = 8
data = np.zeros((dim_x,dim_y))
data.shape

#reshaped_results = results.reshape((dim_x,dim_y,1,19999))

for x in range(dim_x):
    for y in range(dim_y):
        data[x,y] = np.max(np.abs(results[(dim_y*x)+y,:]))

fig, ax = plt.subplots(figsize=(16, 8))

# Place data on map
im = ax.imshow(data.T, cmap='hot')

# Create legend bar
bar = ax.figure.colorbar(im, ax=ax)
bar.ax.set_ylabel("SNR Value", rotation=-90, va="bottom")

# Show tick marks and label them
x_increments = np.arange(dim_x)
ax.set_xticks(x_increments, labels=x_increments)
y_increments = np.arange(dim_y)
ax.set_yticks(y_increments, labels=y_increments)

# # Label axes
ax.set_xlabel("X-Axis")
ax.set_ylabel("Y-Axis")